# **1. Importação de Bibliotecas**


Bibliotecas padrão para manipulação de dados e visualização.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.3 MB/s eta 0:00:00


Ferramentas utilizadas para modelagem de tópicos e redução de dimensionalidade.

In [3]:
from bertopic import BERTopic
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer

Biblioteca para manipulação de linguagem natural, usada aqui para lidar com *stopwords* no português.

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
stopwords_pt = set(stopwords.words('portuguese'))
stopwords_pt=list(stopwords_pt)
vectorizer_model  = CountVectorizer(stop_words=stopwords_pt)

Usado para "montar" o Google Drive no Google Colab, possibilitando acesso a arquivos armazenados nele.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **2. Pré-Processamento**




In [7]:
df=pd.read_csv('/content/drive/MyDrive/Geral.csv', sep='|')
df.head()

,text
0,"Marta tem 6 bolas de ouro, mas ele queria ter ..."
1,Mancada fulminante 😅
2,Sem viés ideológico assisti todos os jogos na ...
3,Querem so lacrar mais nada
4,Eu nem sabia que tava tendo copa do mundo femi...


Substituir as marcações [mention] encontradas no texto por uma string vazia (''), removendo-as.

In [8]:
def remover_tag(texto):
    return texto.replace('[mention]', '')

Decodificar entidades HTML para caracteres normais.


In [9]:
import html
def decode_html_entities(text):
    decoded_text = html.unescape(text)
    return decoded_text


Remover URLs do texto utilizando uma expressão regular.

In [10]:
def remove_links(text):
    link_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))href+="about:invalid#zCSafez'
    text_without_links = re.sub(link_pattern, '', text)

    return text_without_links

Remover palavras ou padrões que representam risadas.


In [11]:
def remove_risada(text):
    padrao_risada = r'\b(?:k{2, }|haha|hehe|hahaha|hehehe|rsrs)\b'
    texto_sem_risada = re.sub(padrao_risada, '', text, flags=re.IGNORECASE)
    return texto_sem_risada

Substituir todos os emojis encontrados no texto por uma string vazia (''), removendo-os.

In [12]:
import re

def caracter_especial_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emojis gerais
                               u"\U0001F300-\U0001F5FF"  # Emojis de símbolos e transportes
                               u"\U0001F700-\U0001F77F"  # Emojis de alquimia
                               u"\U0001F780-\U0001F7FF"  # Emojis de Geometria
                               u"\U0001F800-\U0001F8FF"  # Emojis de sinais gregos
                               u"\U0001F900-\U0001F9FF"  # Emojis de sinais suplementares
                               u"\U0001FA00-\U0001FA6F"  # Emojis de esportes
                               u"\U0001FA70-\U0001FAFF"  # Emojis de comida
                               u"\U0001F680-\U0001F6FF"  # Emojis de transporte terrestre
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text

In [13]:
df['text'] = df['text'].apply(caracter_especial_emoji)

Remover quebras de linha (\n) do texto.

In [14]:
def remove_newlines(text):
    texto_limpo = text.replace("\n", " ")
    return texto_limpo

In [15]:
df['text'] = df['text'].apply(remove_newlines)


In [16]:
df

,text
0,"Marta tem 6 bolas de ouro, mas ele queria ter ..."
1,Mancada fulminante
2,Sem viés ideológico assisti todos os jogos na ...
3,Querem so lacrar mais nada
4,Eu nem sabia que tava tendo copa do mundo femi...
...,...
11918,Começou errado ja no critério de convocação da...
11919,@Raphael Abreu kkkkkkkkkkk se nao empurrar dir...
11920,A bola nao entrou so isso.
11921,Empataram com um time sub 15


Transformar a coluna "text" em uma lista de strings para a análise de tópicos e configuração para rotular os textos.

In [17]:
documents = df['text'].tolist()


In [18]:
custom_headers = ['Document']

In [19]:
df['text']

,text
0,"Marta tem 6 bolas de ouro, mas ele queria ter ..."
1,Mancada fulminante
2,Sem viés ideológico assisti todos os jogos na ...
3,Querem so lacrar mais nada
4,Eu nem sabia que tava tendo copa do mundo femi...
...,...
11918,Começou errado ja no critério de convocação da...
11919,@Raphael Abreu kkkkkkkkkkk se nao empurrar dir...
11920,A bola nao entrou so isso.
11921,Empataram com um time sub 15


# **3. Bertopic**

In [20]:
from bertopic.representation import MaximalMarginalRelevance
from transformers import pipeline
from bertopic.representation import TextGeneration
from bertopic.representation import KeyBERTInspired
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic

In [21]:
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
topic_model = BERTopic(umap_model=umap_model, embedding_model = 'paraphrase-multilingual-mpnet-base-v2', language='multilingual', vectorizer_model  = vectorizer_model  )

Devido à natureza estocástica do UMAP, os resultados do BERTopic podem ser diferentes mesmo que você execute o mesmo código várias vezes. Se quiser reproduzir os resultados, às custas do desempenho, você pode definir um random_state no UMAP para evitar qualquer comportamento estocástico:

In [22]:
from bertopic import BERTopic
from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=5,
                  min_dist=0.0, metric='cosine', random_state=42)
topic_model = BERTopic(umap_model=umap_model)

In [23]:
topics, probs = topic_model.fit_transform(documents)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3254,-1_que_de_no_seleo,"[que, de, no, seleo, com, pra, da, tem, um, do]",[Agora que a marta vai se aposentar agora esqu...
1,0,942,0_feminino_futebol_masculino_assistir,"[feminino, futebol, masculino, assistir, no, d...","[Futebol feminino não dá pra assistir..., Fute..."
2,1,845,1_lacrao_lacradoras_lacra_lacrar,"[lacrao, lacradoras, lacra, lacrar, muita, lac...","[Quem lacra não lucra, Quem lacra não lucra !,..."
3,2,362,2_kkkk_kkkkk_kkk_kkkkkk,"[kkkk, kkkkk, kkk, kkkkkk, kkkkkkk, kkkkkkkk, ...","[ISSO AÍ KKKKK, Que delícia kkkkk, Kkkkkkk sel..."
4,3,360,3_brasil_brasileira_do_brasileiro,"[brasil, brasileira, do, brasileiro, no, que, ...",[@marcos sousa difícil é mais fácil o Brasil g...
...,...,...,...,...,...
125,124,11,124_feminista_bl_feministas_cheia,"[feminista, bl, feministas, cheia, lacrao, pou...","[Muita LACRAÇÃO e COITADISMO FEMINISTA, e pouc..."
126,125,11,125_rapinoe_2019_fala_isso,"[rapinoe, 2019, fala, isso, em, tore, branca, ...","[Fala isso pra Rapinoe em 2019., Fala isso pra..."
127,126,10,126_vergonha_voce_feioas_verhonha,"[vergonha, voce, feioas, verhonha, adversaria,...",[Verdade a mídia de merda Neymar um lixo seleç...
128,127,10,127_13_adorei_adoreichupaaa_londinho,"[13, adorei, adoreichupaaa, londinho, lu, lula...","[Adorei kk, Lula 13, Lula 13]"


# **4. Hierarquização**

Cálculo da matriz de similaridade entre os tópicos, e organização das informações em um formato tabular, identificando os 20 pares de tópicos mais similares.

In [25]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

distance_matrix = cosine_similarity(np.array(topic_model.topic_embeddings_))
dist_df = pd.DataFrame(distance_matrix, columns=topic_model.topic_labels_.values(),
                       index=topic_model.topic_labels_.values())

tmp = []
for rec in dist_df.reset_index().to_dict('records'):
    t1 = rec['index']
    for t2 in rec:
        if t2 == 'index':
            continue
        tmp.append(
            {
                'topic1': t1,
                'topic2': t2,
                'distance': rec[t2]
            }
        )

pair_dist_df = pd.DataFrame(tmp)

pair_dist_df = pair_dist_df[(pair_dist_df.topic1.map(lambda x: not x.startswith('-1'))) &
                            (pair_dist_df.topic2.map(lambda x: not x.startswith('-1')))]
pair_dist_df = pair_dist_df[pair_dist_df.topic1 < pair_dist_df.topic2]
pair_dist_df.sort_values('distance', ascending=False).head(20)

,topic1,topic2,distance
15436,117_concordo_literalmente_pilhado_com,95_plenamente_concordo_totalmente_pilhado,0.945628
4689,35_masculina_feminina_seleo_imagina,8_feminina_seleo_essa_selecao,0.939813
7029,53_feminina_pilhado_seleo_orgasmo,8_feminina_seleo_essa_selecao,0.932327
15781,120_copa_feminina_europa_lagrima,50_copa_mundo_feminina_tendo,0.930657
2605,19_brasileiro_futebol_brasil_pas,4_futebol_pouco_muita_nada,0.929139
15395,117_concordo_literalmente_pilhado_com,54_concordo_100_com_pilhado,0.927613
13425,102_copa_ganha_copas_1914e,34_copa_mundo_amrica_copas,0.926021
1439,10_brasileira_feminina_brasil_seleo,8_feminina_seleo_essa_selecao,0.922485
15652,119_kkkkkkkkkk_kkkkkkkkkkkkkk_kkkkkkkkkkkk_kkk...,51_kkkkkkkkk_kkkkkkkkkkk_kkkkk_kkkkkkk,0.920070
4040,30_deixa_falar_ningum_ninguem,9_pilhado_esse_foda_chato,0.917776


Mesclagem

In [26]:
topic_model.merge_topics(df['text'], [[117, 95, 54], [35, 8, 53, 10, 65], [120, 50, 10], [19, 4], [102, 34], [119, 51], [30, 9], [6, 94, 16], [126, 48], ])
df['merged_topic'] = topic_model.topics_

In [27]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3254,-1_que_de_no_seleo,"[que, de, no, seleo, com, pra, da, tem, um, do]",[Agora que a marta vai se aposentar agora esqu...
1,0,942,0_feminino_futebol_masculino_no,"[feminino, futebol, masculino, no, assistir, d...","[Futebol feminino não dá pra assistir..., Fute..."
2,1,845,1_lacrao_lacradoras_lacra_lacrar,"[lacrao, lacradoras, lacra, lacrar, muita, lac...","[Quem lacra não lucra , Quem lacra não lucra ,..."
3,2,401,2_jamaica_frana_vaquinha_pra,"[jamaica, frana, vaquinha, pra, da, brasil, pa...",[Jamaica fez vaquinha pra ir pra copa do mundo...
4,3,377,3_futebol_brasileiro_pouco_nada,"[futebol, brasileiro, pouco, nada, no, muita, ...","[Futebol nada, Brasil não é mais o país do fut..."
...,...,...,...,...,...
111,110,11,110_rapinoe_2019_fala_isso,"[rapinoe, 2019, fala, isso, em, pra, tore, bra...","[Fala isso pra Rapinoe em 2019., Fala isso pra..."
112,111,11,111_feminista_bl_feministas_cheia,"[feminista, bl, feministas, cheia, lacrao, pou...","[Muita LACRAÇÃO e COITADISMO FEMINISTA, e pouc..."
113,112,11,112_hrefaboutinvalidzcsafezaa_hrefaboutinvalid...,"[hrefaboutinvalidzcsafezaa, hrefaboutinvalidzc...","[A Pia é uma grande furada, o barco naufragou!..."
114,113,10,113_13_adorei_adoreichupaaa_londinho,"[13, adorei, adoreichupaaa, londinho, lu, lula...","[Adorei kk, Lula 13, Lula 13]"


In [28]:
custom_headers = ['Document']

Hierarquia

In [29]:
hierarchical_topics = topic_model.hierarchical_topics(df['text'])

100%|██████████| 114/114 [00:00<00:00, 228.86it/s]


In [30]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

Redução

In [31]:
topic_model.merge_topics(df['text'], [[56, 43, 52], [92, 62, 65, 47], [39, 13, 18, 46], [21, 1, 26], [24, 6, 9, 5, 10, 0, 3, 22], [27, 17], [100, 40, 83], [68, 91, 29, 57], [104, 70, 28, 51], [86, 20, 19, 33], [30, 35, 11, 16, 12, 67, 58, 73], [99, 98], [89, 55, 93, 80], [53, 111], [63, 23, 97], [36, 50], [110, 101, 96, 113, 105, 103, 31, 25, 99, 95, 109, 74], [15, 84, 77, 32, 38], [85, 14, 69, 102, 8, 114], [106, 37, 41, 61, 49], [48, 60, 54, 87, 82, 59, 44], [42, 79, 2, 45, 71], [81, 7, 72, 66, 4, 34, 75, 64, 112, 94], [78, 108, 107, 88]])
df['merged_topic'] = topic_model.topics_

In [32]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,3254,-1_que_de_no_seleo,"[que, de, no, seleo, da, com, do, pra, tem, um]",[Agora que a marta vai se aposentar agora esqu...
1,0,2607,0_futebol_feminino_no_que,"[futebol, feminino, no, que, de, brasil, seleo...",[foi FEIO e tbm humilhante a desclassificação?...
2,1,994,1_lacrao_lacradoras_lacrar_lacra,"[lacrao, lacradoras, lacrar, lacra, muita, fut...","[MUITA LACRAÇÃO E POUCO FUTEBOL, É MUITA LACRA..."
3,2,812,2_kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk_hrefa...,"[kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk, hrefa...","[Kkkk, kkkk, Kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk..."
4,3,655,3_neto_neymar_marta_panam,"[neto, neymar, marta, panam, no, foi, do, ganh...","[Neto está há 0 dias sem falar do Neymar , Net..."
5,4,521,4_jamaica_da_pra_no,"[jamaica, da, pra, no, que, seleo, para, de, b...","[Parabéns Jamaica , Parabéns JAMAICA, Jamaica ..."
6,5,450,5_vampeta_pilhado_deixa_falar,"[vampeta, pilhado, deixa, falar, esse, cara, f...","[Pilhado não deixa ninguém falar, Deixa o Vamp..."
7,6,316,6_meninas_criticar_as_que,"[meninas, criticar, as, que, no, tem, pilhado,...",[Essas meninas nunca jogaram nada o Neto está ...
8,7,272,7_prado_flvio_retrgrados_desprezam,"[prado, flvio, retrgrados, desprezam, buscando...",[É uma vergonha o comportamento MACHISTA de jo...
9,8,269,8_pilhado_certo_esporte_gritinhos,"[pilhado, certo, esporte, gritinhos, trombadas...","[ TROMBADAS, GRITINHOS e CHITES AO AR !! Essa..."


In [33]:
topic_model.merge_topics(df['text'], [[-1, 2], [5, 8, 10, 13, 24], [6, 18, 19, 21], [17, 20], [22, 25]])
df['merged_topic'] = topic_model.topics_

In [34]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4066,-1_que_de_no_seleo,"[que, de, no, seleo, da, com, do, pra, tem, um]","[Essa seleção feminina, aquela seleção do Neym..."
1,0,2607,0_futebol_no_feminino_que,"[futebol, no, feminino, que, de, brasil, seleo...","[Vampeta tentando justificar o injustificável,..."
2,1,1068,1_pilhado_vampeta_concordo_deixa,"[pilhado, vampeta, concordo, deixa, falar, fal...","[Parabéns pilhado., E o Pilhado também, Pilhad..."
3,2,994,2_lacrao_de_lacradoras_futebol,"[lacrao, de, lacradoras, futebol, seleo, lacra...","[Muita lacração e pouco futebol!, Muita lacraç..."
4,3,655,3_neto_neymar_marta_no,"[neto, neymar, marta, no, panam, do, foi, de, ...","[Neto está há 0 dias sem falar do Neymar , Net..."
5,4,544,4_meninas_criticar_que_no,"[meninas, criticar, que, no, de, tem, vergonha...",[Temos que torcer para as meninas da seleção ....
6,5,521,5_jamaica_da_no_que,"[jamaica, da, no, que, pra, seleo, de, para, d...","[Parabéns Jamaica , Jamaica ✳️✳️✳️, Jamaica!!!..."
7,6,272,6_prado_flvio_retrgrados_desprezam,"[prado, flvio, retrgrados, desprezam, buscando...",[É uma vergonha o comportamento MACHISTA de jo...
8,7,236,7_treinadora_pia_cristiane_no,"[treinadora, pia, cristiane, no, elias, essa, ...","[PÍFIA A PIA RIDÍCULO, *Pia, CRISTIANE náo foi..."
9,8,186,8_seleo_masculina_essa_que,"[seleo, masculina, essa, que, bosta, de, fraca...","[A seleção masculina um bagaço também, Seleção..."


Extrair todos os documentos de cada tópico (arquivo csv)

In [38]:
doc_info = topic_model.get_document_info(documents)

from collections import defaultdict

topic_docs = defaultdict(list)
for index, row in doc_info.iterrows():
    topic = row['Topic']
    doc = row['Document']
    topic_docs[topic].append(doc)

target_topic = 0
documents_in_topic_0 = topic_docs[target_topic]
print(documents_in_topic_0)

topic_0_df = pd.DataFrame(documents_in_topic_0, columns=["Document"])

topic_0_file_name = "documents_in_topic_0.csv"

topic_0_df.to_csv(topic_0_file_name, index=False)

print(f"Documents in topic 0 saved to {topic_0_file_name}")

['Sem viés ideológico assisti todos os jogos na TV caze não na rede esgoto que final parabéns Espanha e Inglaterra que espetáculo. Quanto ao fiasco do Brasil normal seja masculino e feminino', 'Eu nem sabia que tava tendo copa do mundo feminino ', 'O problema do Brasil foi a Bola', 'Brasil não e mais pais do futebol, Brasil masculino fudido, brasil sub 20 fudido, brasil feminino fudido tmb, ta tudo uma merda kkkkkkkkkkkkkkkkk', 'País do futebol nunca deixou de ser a Inglaterra bro.', 'Realmente essa seleção feminina só tem papo. Sempre com esse negócio de apoio. Uma vergonha.', 'Time do Brasil feminino perde até para deficiente jogando de muletas.', 'Futebol feminino tem que ser proibido, ridículo', 'Tem que fazer teste hormonal pra saber se realmente e feminina ou não', 'Está doido neto ? Neymar nos representa sim nos a maioria ainda amamos o futebol quem nao me representa é essa seleção, as jogadoras que de fato suaram pra dar nome e posição para esse futebol não foram chamadas a que

In [40]:
doc_info = topic_model.get_document_info(documents)

from collections import defaultdict

topic_docs = defaultdict(list)
for index, row in doc_info.iterrows():
    topic = row['Topic']
    doc = row['Document']
    topic_docs[topic].append(doc)

target_topic = 4
documents_in_topic_4 = topic_docs[target_topic]

topic_4_df = pd.DataFrame(documents_in_topic_4, columns=["Document"])

topic_4_file_name = "documents_in_topic_4.csv"

topic_4_df.to_csv(topic_4_file_name, index=False)

print(f"Documents in topic 4 saved to {topic_4_file_name}")

Documents in topic 4 saved to documents_in_topic_4.csv


Extrair os documentos *Outliers*

In [47]:
outliers = doc_info[doc_info['Topic'] == -1]

outlier_file_name = "outlier_documents.csv"
outliers.to_csv(outlier_file_name, index=False)

print(f"Outlier documents saved to {outlier_file_name}")

Outlier documents saved to outlier_documents.csv


# **Visualizações**

Gráfico de barras que exibe os tópicos mais relevantes e suas palavras-chave.

In [48]:
topic_model.visualize_barchart(top_n_topics = 15, n_words = 20)

Matriz que indica a semelhança de determinados tópicos entre si

In [49]:
topic_model.visualize_heatmap(n_clusters=8)

Informações gerais sobre o tópico, inclusive seu tamanho e suas palavras correspondentes

In [50]:
topic_model.visualize_topics()


Para mais informações, consulte: [https://maartengr.github.io/BERTopic/index.html](https://)